In [1]:
import cv2
import itertools
import numpy as np
from time import time
import mediapipe as mp
import matplotlib.pyplot as plt
mp_face_mesh=mp.solutions.face_mesh
face_mesh=mp_face_mesh.FaceMesh()
from playsound import playsound


In [2]:
def processing(final_image):

    img_gray = cv2.cvtColor(final_image, cv2.COLOR_BGR2GRAY)
    # Blur the image for better edge detection
    img_blur = cv2.GaussianBlur(img_gray, (3,3), 0) 

    # Sobel Edge Detection
    sobelx = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
    sobely = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
    sobelxy = cv2.Sobel(src=img_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection

    # Canny Edge Detection
    edges = cv2.Canny(image=img_gray, threshold1=30, threshold2=50) # Canny Edge Detection
    # Display Canny Edge Detection Image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
    img = cv2.morphologyEx(edges, cv2.MORPH_DILATE, kernel)
    return img

In [3]:
from keras.models import load_model
model=load_model('C:\\Users\\msi1\\Downloads\\Trained_model_softbalanced.h5')
cap=cv2.VideoCapture(0)
counter=0
while cap.isOpened():   
        ret,frame=cap.read()
        rgb_image=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image_height, image_width, _ = frame.shape
        result= face_mesh.process(rgb_image)
        # Check if facial landmarks in the image are found.
        if result.multi_face_landmarks:
            # Iterate over the found faces.
            for face_landmarks in result.multi_face_landmarks:
                LEFT_EYE_INDEXES = list(set(itertools.chain(*mp_face_mesh.FACEMESH_LEFT_EYE)))
                RIGHT_EYE_INDEXES = list(set(itertools.chain(*mp_face_mesh.FACEMESH_RIGHT_EYE)))
                INDEXES = mp_face_mesh.FACEMESH_LEFT_EYE
                print(INDEXES)
                INDEXES_LIST = list(itertools.chain(*INDEXES))
                print(INDEXES_LIST)
                landmarts = []
                for INDEX in INDEXES_LIST:
                    landmarts.append([int(face_landmarks.landmark[INDEX].x * image_width),
                                       int(face_landmarks.landmark[INDEX].y * image_height)])
                    f, d, width, height = cv2.boundingRect(np.array(landmarts))
                    landmarks = np.array(landmarts)
                INDEXES2 = mp_face_mesh.FACEMESH_RIGHT_EYE
                print(INDEXES2)
                INDEXES_LIST2 = list(itertools.chain(*INDEXES2))
                print(INDEXES_LIST2)
                landmarts2 = []
                for INDEX2 in INDEXES_LIST2:
                    landmarts2.append([int(face_landmarks.landmark[INDEX2].x * image_width),
                                       int(face_landmarks.landmark[INDEX2].y * image_height)])
                    f2, d2, width2, height2 = cv2.boundingRect(np.array(landmarts2))
                    landmarks = np.array(landmarts2)
            cv2.rectangle(frame,(f2-5,d2-15),(f2+width2+13,d2+height2+15),(0,255,0),2)
            ROI=frame[d-15:d+height+15,f-5:f+width+13]
            final_image= cv2.resize(ROI,(100,100))
            final_image=processing(final_image)
            model=load_model('C:\\Users\\msi1\\Downloads\\Trained_model_softbalanced.h5')
            img=final_image/255.0
            img = np.expand_dims(img, axis=0)
            img=img.reshape(img.shape[0], img.shape[1], img.shape[2],1)
            pred=model.predict(img) 
            if pred[0][0]<pred[0][1]:
                status="open eyes"
                cv2.putText(frame,'Active',(0,150),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,255),2)
                counter=0
            else:
            
                counter=counter+1
                status="Drowsy"
                cv2.putText(frame,status,(0,150),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,255),2)
                cv2.putText(frame,str(counter),(300,100), cv2.FONT_HERSHEY_TRIPLEX,2,(0,0,255),2)
                
                if counter>20:
                       
                 #cv2.putText(frame,'warning',(0,150),cv2.FONT_HERSHEY_SIMPLEX,3,(0,0,255),2,cv2.LINE_4)
                    cv2.putText(frame, "warning", (0,400),cv2.FONT_HERSHEY_SIMPLEX, 3,(255,0,0), 2)
                    playsound("C:\\Users\\msi1\\Desktop\\beep.wav")
                
                    counter=0
                    
                
                
                
                    
                    
                    

            
            cv2.rectangle(frame,(f-5,d-15),(f+width+13,d+height+15),(0,255,0),2)
        
           
            cv2.imshow('Drowsiness Detection system', frame)
            if cv2.waitKey(5) & 0xFF == ord("q"):
                break
            #cv2.waitKey(0) 
            #cv2.imshow('MediaPipe FaceMes', final_image)
            #cv2.waitKey(0)

cap.release()
cv2.destroyAllWindows()

frozenset({(374, 380), (390, 373), (249, 390), (385, 384), (373, 374), (263, 466), (387, 386), (381, 382), (384, 398), (466, 388), (386, 385), (388, 387), (263, 249), (380, 381), (398, 362), (382, 362)})
[374, 380, 390, 373, 249, 390, 385, 384, 373, 374, 263, 466, 387, 386, 381, 382, 384, 398, 466, 388, 386, 385, 388, 387, 263, 249, 380, 381, 398, 362, 382, 362]
frozenset({(154, 155), (33, 7), (246, 161), (159, 158), (144, 145), (173, 133), (158, 157), (163, 144), (153, 154), (7, 163), (161, 160), (157, 173), (33, 246), (160, 159), (145, 153), (155, 133)})
[154, 155, 33, 7, 246, 161, 159, 158, 144, 145, 173, 133, 158, 157, 163, 144, 153, 154, 7, 163, 161, 160, 157, 173, 33, 246, 160, 159, 145, 153, 155, 133]
frozenset({(374, 380), (390, 373), (249, 390), (385, 384), (373, 374), (263, 466), (387, 386), (381, 382), (384, 398), (466, 388), (386, 385), (388, 387), (263, 249), (380, 381), (398, 362), (382, 362)})
[374, 380, 390, 373, 249, 390, 385, 384, 373, 374, 263, 466, 387, 386, 381, 38